We see that there are many missing values and they can be taken from the internet. I can proceed in that direction.

In [1]:
import pandas as pd
import numpy as np
import mysql.connector

In [2]:
df_netflix_movie_data = pd.read_csv('./Datasets/netflix_titles.csv', lineterminator = '\n')
df_prime_movie_data = pd.read_csv('./Datasets/amazon_prime_titles.csv', lineterminator = '\n')
df_disney_plus_movie_data = pd.read_csv('./Datasets/disney_plus_titles.csv', lineterminator = '\n')
df_hulu_movie_data = pd.read_csv('./Datasets/hulu_titles.csv', lineterminator = '\n')

In [ ]:
missing_value_netflix = df_netflix_movie_data.isna().sum()
print(missing_value_netflix)

In [ ]:
print("Number of records in Netflix DataFrame:", df_netflix_movie_data.shape[0])

In [ ]:
missing_value_prime = df_prime_movie_data.isna().sum()
print(missing_value_prime)

In [ ]:
print("Number of records in Prime Video DataFrame:", df_prime_movie_data.shape[0])

In [ ]:
missing_value_disney_plus = df_disney_plus_movie_data.isna().sum()
print(missing_value_disney_plus)

In [ ]:
print("Number of records in Disney Plus DataFrame:", df_disney_plus_movie_data.shape[0])

In [ ]:
missing_value_hulu = df_hulu_movie_data.isna().sum()
print(missing_value_hulu)

In [ ]:
print("Number of records in Hulu DataFrame:", df_hulu_movie_data.shape[0])

As we can see clearly, above only columns director, cast, country, date_added, rating, duration. Rather than deleting them. If we can get those values back in the dataset it would be best.
I can use information present on the website imdb to get the data and fill it. I can use web scrapping to get the data and fill it in the dataset.

But before that I can check data. One dataset row may have data of the for the other dataset missing row values. Let's first check for the 

In [ ]:
print("Netflix DataFrame Column Data Types:")
print(df_netflix_movie_data.dtypes)

In [ ]:
print("\nPrime Video DataFrame Column Data Types:")
print(df_prime_movie_data.dtypes)

In [ ]:

print("\nDisney Plus DataFrame Column Data Types:")
print(df_disney_plus_movie_data.dtypes)

In [ ]:
print("\nHulu DataFrame Column Data Types:")
print(df_hulu_movie_data.dtypes)

Okay I will be first converting the datatypes of the columns of the dataframes according to the requirement and then after that I will be tranferring data to the SQL. Through this I can ensure that there is no error while tranferring the data.

Why I want to transfer the data to SQL?
I currently want to fill the missing values of the columns director, cast, country, date_added, rating and duration in all datasets. But there are chances that missing value of 1 dataset might be present in another dataset. So for doing that in optimized manner I am using the SQL. 
Other approaches:

- I can loop through datasets to find values. But this approach will take most time as there are 4 datasets and time taken will get to O($n^4$). This approach is not great.
- I can use Hash tables for this but it will be take most of the memory for it. It would be better to go for Database as I would be taking it's help for data wrangling also.

In [3]:
dataframes = [df_netflix_movie_data, df_prime_movie_data, df_disney_plus_movie_data, df_hulu_movie_data]

In [4]:
for df in dataframes:
    df["show_id"] = df["show_id"].astype("string")  # Explicit string type
    df["type"] = df["type"].astype("string")
    df["title"] = df["title"].astype("string")
    df["director"] = df["director"].astype("string")
    df["cast"] = df["cast"].astype("string")  # Convert float64 to string
    df["country"] = df["country"].astype("string")
    df["date_added"] = pd.to_datetime(df["date_added"], errors="coerce")  # Convert to Date
    df["release_year"] = df["release_year"].astype("Int64")  # Explicitly use Int64
    df["rating"] = df["rating"].astype("string")
    df["duration"] = df["duration"].astype("string")
    df["listed_in"] = df["listed_in"].astype("string")
    df["description"] = df["description"].astype("string")



In [5]:
# Verify Data Types
for i, df in enumerate(dataframes, start=1):
    print(f"\nData Types for DataFrame {i}:\n", df.dtypes)


Data Types for DataFrame 1:
 show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object

Data Types for DataFrame 2:
 show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast            string[python]
country         string[python]
date_added      datetime64[ns]
release_year             Int64
rating          string[python]
duration        string[python]
listed_in       string[python]
description     string[python]
dtype: object

Data Types for DataFrame 3:
 show_id         string[python]
type            string[python]
title           string[python]
director        string[python]
cast          

In [6]:
conn = mysql.connector.connect(
    host="127.0.0.1",
    user="Himanshu",
    password="Hustling@2000",
)
cursor = conn.cursor()

In [7]:
cursor.execute("DROP DATABASE IF EXISTS movies_db")
conn.commit()

In [8]:
cursor.execute("CREATE DATABASE IF NOT EXISTS movies_db;")
cursor.close()
conn.close()

In [9]:
conn = mysql.connector.connect(
    host="localhost",
    user="Himanshu",
    password="Hustling@2000",
    database="movies_db"
)
cursor = conn.cursor()

In [10]:
cursor.execute("""
    CREATE TABLE IF NOT EXISTS movies_data (
        show_id VARCHAR(255),
        type VARCHAR(50),
        title VARCHAR(500),
        director VARCHAR(300),
        cast VARCHAR(2000),
        country VARCHAR(100),
        date_added DATE,
        release_year YEAR,
        rating VARCHAR(50),
        duration VARCHAR(100),
        listed_in VARCHAR(300),
        description TEXT,
        PRIMARY KEY (title, release_year)
    )
""")

Okay so the database has been created. But I have to make sure that all the data goes properly inside the database.

In [ ]:
# Initialize failed_inserts as an empty DataFrame with correct columns
df_failed_inserts = pd.DataFrame(columns=df.columns)

# Assuming you have already established a connection and cursor
cursor = conn.cursor()

# Insert each row, perform verification, and handle errors
for _, row in df.iterrows():
    try:
        # Convert NaN values to None, which can be handled by MySQL
        row = row.where(pd.notna(row), None)

        # Log the row that is about to be inserted
        print(f"\nInserting row with title: {row['title']} and release_year: {row['release_year']}")

        # Insert data into the database
        insert_query = """
            INSERT INTO movies_data 
            (show_id, type, title, director, cast, country, date_added, release_year, 
             rating, duration, listed_in, description)
            VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
        """
        cursor.execute(insert_query, (
            row["show_id"], row["type"], row["title"], row["director"],
            row["cast"], row["country"], row["date_added"], row["release_year"],
            row["rating"], row["duration"], row["listed_in"], row["description"]
        ))
        conn.commit()  # Commit the transaction

        # Log success
        print(f"Data for '{row['title']}' inserted successfully.")

    except mysql.connector.Error as err:
        # Log the error
        print(f"\nError occurred while inserting data: {err}")
        print(f"Row causing the error: \n{row.to_frame().T}")  # Print row in tabular format

        # Append failed row properly to df_failed_inserts
        df_failed_inserts = pd.concat([df_failed_inserts, row.to_frame().T], ignore_index=True)

# Print failed insertions, if any
if not df_failed_inserts.empty:
    print("\nFailed Insertions due to duplicates or errors: ")
    print(df_failed_inserts)
else:
    print("\nAll rows inserted successfully without errors.")

In [ ]:
display(df_failed_inserts)

In [14]:
cursor = conn.cursor()

# Function to update the database with missing values filled
def update_missing_values(row):
    try:
        # Extract the row data from the database using Title and Release Year
        cursor.execute("""
            SELECT * FROM movies_data WHERE title = %s AND release_year = %s
        """, (row["title"], row["release_year"]))
        
        db_row = cursor.fetchone()

        if db_row:
            # Convert fetched row into a dictionary
            columns = ["show_id", "type", "title", "director", "cast", "country", 
                       "date_added", "release_year", "rating", "duration", 
                       "listed_in", "description"]
            db_data = dict(zip(columns, db_row))

            # Compare each element and fill missing values
            updated_data = {}
            for col in columns:
                if pd.isna(row[col]) or row[col] is None:
                    updated_data[col] = db_data[col]  # Fill missing value from DB
                else:
                    updated_data[col] = row[col]  # Keep the original value if not missing
            
            # If any values were updated, modify the database
            update_query = """
                UPDATE movies_data
                SET show_id = %s, type = %s, director = %s, cast = %s, country = %s, 
                    date_added = %s, rating = %s, duration = %s, listed_in = %s, 
                    description = %s
                WHERE title = %s AND release_year = %s
            """
            cursor.execute(update_query, (
                updated_data["show_id"], updated_data["type"], updated_data["director"], 
                updated_data["cast"], updated_data["country"], updated_data["date_added"], 
                updated_data["rating"], updated_data["duration"], updated_data["listed_in"], 
                updated_data["description"], updated_data["title"], updated_data["release_year"]
            ))
            conn.commit()
            print(f"Updated missing values for: {row['title']} ({row['release_year']})")
        else:
            print(f"No matching record found in database for: {row['title']} ({row['release_year']})")

    except mysql.connector.Error as err:
        print(f"Error updating {row['title']} ({row['release_year']}): {err}")

# Loop through each row in the dataframe and process
for _, row in df_failed_inserts.iterrows():
    update_missing_values(row)

print("Data processing complete.")

Updated missing values for: Trolls: TrollsTopia (2020)
No matching record found in database for: Monos (2019)
Data processing complete.
